In [21]:
import pandas as pd
import numpy as np

In [39]:
'''Секунды в строку ЧЧ:ММ:СС, на входе скалярно'''
def seconds_to_str(seconds):
    mm, ss = divmod(seconds, 60)
    hh, mm = divmod(mm, 60)
    return "%02d:%02d:%02d" % (hh, mm, ss)

In [22]:
datadir='data/'
name='RetailTrans.csv'
data=pd.read_csv(datadir+name)
data.sort_values(by='Date', ignore_index=True, inplace=True)
data

,TransactionID,Date,CustomerName,Product,TotalItems,TotalCost,PaymentMethod,City,StoreType,DiscountApplied,CustomerCategory,Season,Promotion,Numprod
0,N1000526511,2020-01-01 00:03:54,Diane Fleming,Broom,5,90.28,Debit Card,Houston,Supermarket,True,Middle-Aged,Fall,No_promo,4
1,N1000589096,2020-01-01 00:07:48,Larry Roberts,Bread,1,10.69,Cash,Miami,Convenience Store,False,Retiree,Spring,BOGO (Buy One Get One),3
2,N1000620889,2020-01-01 00:08:42,Scott Li,Bath Towels,4,67.54,Credit Card,New York,Warehouse Club,True,Teenager,Winter,No_promo,2
3,N1000579003,2020-01-01 00:09:02,Jennifer Edwards,Ketchup,1,13.39,Debit Card,Miami,Warehouse Club,False,Professional,Summer,Discount on Selected Items,4
4,N1000969745,2020-01-01 00:10:25,Jason Patterson,Mop,9,50.32,Cash,Houston,Supermarket,True,Young Adult,Winter,No_promo,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,N1000862966,2024-05-18 19:26:29,Emily Melendez,Razors,5,70.85,Credit Card,Los Angeles,Pharmacy,False,Middle-Aged,Summer,Discount on Selected Items,4
999996,N1000442620,2024-05-18 19:28:08,Devon Hooper,Soap,10,86.27,Mobile Payment,Boston,Pharmacy,True,Teenager,Summer,BOGO (Buy One Get One),1
999997,N1000446202,2024-05-18 19:29:12,Carl Howe,Cleaning Rags,2,5.06,Mobile Payment,Dallas,Department Store,True,Young Adult,Fall,No_promo,3
999998,N1000008443,2024-05-18 19:29:37,Abigail Villarreal,Toothbrush,5,6.22,Mobile Payment,Atlanta,Department Store,True,Retiree,Spring,Discount on Selected Items,3


In [23]:
dt=pd.to_datetime(data.Date)
data['Date']=pd.to_datetime(dt.dt.date)
data['time']=dt.dt.time

In [24]:
from src.colinv import profile_dataframe
import json

results = profile_dataframe(data)
types=pd.json_normalize(results)
types

,column_index,column_name,inferred_type,semantic_sense,semantic_confidence,n_unique_if_categorical,numeric_subtype
0,0,TransactionID,categorical,other,1.000,1000000.0,NaN
1,1,Date,categorical,datetime,1.000,1600.0,NaN
2,2,CustomerName,categorical,None,0.000,329738.0,NaN
3,3,Product,categorical,other,0.625,81.0,NaN
4,4,TotalItems,numeric,None,0.000,NaN,integer
5,5,TotalCost,numeric,other,0.993,NaN,float
6,6,PaymentMethod,categorical,other,0.247,4.0,NaN
7,7,City,categorical,other,0.710,10.0,NaN
8,8,StoreType,categorical,other,0.347,6.0,NaN
9,9,DiscountApplied,boolean,discountapplied,1.000,2.0,NaN


In [25]:
data.columns

Index(['TransactionID', 'Date', 'CustomerName', 'Product', 'TotalItems',
       'TotalCost', 'PaymentMethod', 'City', 'StoreType', 'DiscountApplied',
       'CustomerCategory', 'Season', 'Promotion', 'Numprod', 'time'],
      dtype='object')

In [26]:
numcol=types[types.inferred_type=='numeric'].column_name.values
for c in numcol:
    l=len(data[data[c].isna()])
    if l>0:
        data[c]=data[c].fillna(value='-1.')
    print(c, l)

TotalItems 0
TotalCost 0
Numprod 0


In [27]:
dt=dict(types.inferred_type.value_counts())
dt

{'categorical': 11, 'numeric': 3, 'boolean': 1}

In [28]:
onehot_cols = types[(types.inferred_type=='categorical')&(types.n_unique_if_categorical<=111)].column_name.values
onehot_cols = np.concatenate([onehot_cols, types[(types.inferred_type=='boolean')].column_name.values])
cat_feat_names = types[(types.inferred_type=='categorical')&(types.n_unique_if_categorical>=15)].column_name.values
num_feat_names = types[types.inferred_type=='numeric'].column_name.values
log1p_transform_cols = []
date_feature = 'Date'
time_feature = ''
client_id = 'CustomerName'
mcc_name = 'Product'

latent_dim = {'onehot': len(onehot_cols), 'categorical': len(cat_feat_names), 'numerical': len(num_feat_names)}
latent_dim['cv']=sum(latent_dim.values())
latent_dim 

{'onehot': 8, 'categorical': 5, 'numerical': 3, 'cv': 16}

In [29]:
from time import time

In [30]:
from src.embedder import embeddings
s=time()
X_emb, X_oh, cond_vector, synth_date, scaler_cat, scaler_onehot, scaler_num, cv_params, scaler, round_array = \
            embeddings(data, cat_feat_names, num_feat_names, onehot_cols, date_feature, time_feature, client_id, latent_dim, device='cpu', load=False, epochs=40) #40 epochs
t=time()-s
print(seconds_to_str(t))

Loss E_oh: 0.06102766841650009: 100%|██████████| 40/40 [31:00<00:00, 46.51s/it]
/home/antonij/.config/jupyterlab-desktop/jlab_server/lib/python3.12/site-packages/rdt/transformers/categorical.py:132: FutureWarning: The behavior of value_counts with object-dtype is deprecated. In a future version, this will *not* perform dtype inference on the resulting index. To retain the old behavior, use `result.index = result.index.infer_objects()`
  freq = data.value_counts(normalize=True, dropna=False)
Loss E_num: 1.2687931985055911e-06: 100%|██████████| 40/40 [18:28<00:00, 27.71s/it]


Optimizing poisson intensity...


Loss E_cv: 0.000055061: 100%|██████████| 20/20 [08:50<00:00, 26.51s/it]


4912.165692090988


In [31]:
from src.embedder import train
s=time()
dim_noise = 20
generator, supervisor, loss_array = train(X_emb, cond_vector, latent_dim, dim_noise, epochs=40, load=False)
t=time()-s
print(seconds_to_str(t))

Discriminator Loss: -0.05060 || Discriminator 2 Loss: 0.00016 || Generator Loss: 0.02664 || Supervisor Loss: 0.00980: 100%|██████████| 40/40 [20:48<00:00, 31.20s/it]  

1248.129135608673


In [32]:
from src.generator import sample, inverse_transform
s=time()
n_samples = len(X_emb[:])
synth_data, synth_date, params = sample(n_samples, generator, supervisor, dim_noise, cond_vector, X_emb, cv_params['encoder'], data,\
                                date_feature, client_id, cv_params=cv_params, device='cpu')
t=time()-s
print(seconds_to_str(t))

70.92055177688599


In [41]:
seconds_to_str(4912.165692090988), seconds_to_str(1248.129135608673), seconds_to_str(70.92055177688599)

('01:21:52', '00:20:48', '00:01:10')

In [33]:
X_emb1 = scaler.inverse_transform(X_emb)
synth_data = scaler.inverse_transform(synth_data)

In [34]:
synth_data

array([[ 0.99321485, -0.9228728 ,  0.25439224, ...,  0.8188968 ,
        -0.56332266,  0.85390276],
       [ 0.9883506 , -0.9988671 , -0.27912796, ...,  0.821554  ,
        -0.56514066,  0.8558038 ],
       [ 0.9880001 , -0.99799204,  0.79370064, ...,  0.819766  ,
        -0.5644439 ,  0.8545116 ],
       ...,
       [ 0.97810405, -0.9024814 ,  0.17694584, ..., -0.88464165,
         0.9581818 , -0.61233217],
       [ 0.9980017 , -0.99799883,  0.04905253, ..., -0.8831942 ,
         0.95753664, -0.6104112 ],
       [ 0.998746  , -0.9991688 , -0.33272243, ..., -0.8830063 ,
         0.95795846, -0.6086299 ]], dtype=float32)

In [35]:
import sys
num_feat_names
data[num_feat_names].apply(lambda x: round(x, sys.float_info.dig - 1))

,TotalItems,TotalCost,Numprod
0,5,90.28,4
1,1,10.69,3
2,4,67.54,2
3,1,13.39,4
4,9,50.32,4
...,...,...,...
999995,5,70.85,4
999996,10,86.27,1
999997,2,5.06,3
999998,5,6.22,3


In [36]:
synth_df = inverse_transform(synth_data, latent_dim, X_oh.columns, scaler_onehot, scaler_cat, scaler_num, cat_feat_names,
                             mcc_name, num_feat_names, True, synth_date, time_feature, round_array)

In [42]:
synth_df

,Product,PaymentMethod,City,StoreType,CustomerCategory,Season,Promotion,DiscountApplied,TransactionID,Date,CustomerName,Product,time,TotalItems,TotalCost,Numprod,Date
0,Air Freshener,Credit Card,Atlanta,Convenience Store,Professional,Spring,No_promo,True,N1000975576,1697587200000000000,Kristen Pitts,Garden Hose,10:09:22,9,89.30,5,2042-01-24 00:00:00.000000000
1,Air Freshener,Credit Card,Atlanta,Convenience Store,Homemaker,Spring,Discount on Selected Items,False,N1000648769,1579219200000000000,Joseph Owens,Broom,08:56:30,1,5.48,1,1683-04-28 00:25:26.290448384
2,Air Freshener,Mobile Payment,Atlanta,Warehouse Club,Senior Citizen,Spring,No_promo,True,N1000006893,1706572800000000000,Eric Patrick,Plant Fertilizer,07:38:27,10,93.30,5,2062-06-25 00:00:00.000000000
3,Air Freshener,Credit Card,Atlanta,Specialty Store,Retiree,Fall,No_promo,True,N1000085100,1671840000000000000,Erin Clarke,Razors,08:17:04,7,70.17,4,2026-09-10 00:00:00.000000000
4,Air Freshener,Credit Card,Atlanta,Convenience Store,Student,Winter,BOGO (Buy One Get One),True,N1000111749,1711756800000000000,Mr. Matthew Dodson,Coffee,12:02:39,10,97.07,5,2100-03-04 00:00:00.000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,Air Freshener,Credit Card,Boston,Warehouse Club,Young Adult,Spring,Discount on Selected Items,False,N1000413429,1631491200000000000,Shannon Miller,Light Bulbs,00:44:44,4,42.85,2,2022-01-29 00:00:00.000000000
999996,Air Freshener,Mobile Payment,Atlanta,Warehouse Club,Young Adult,Spring,BOGO (Buy One Get One),True,N1000660301,1594684800000000000,Lauren Smith,Dish Soap,07:49:18,2,16.92,1,2020-07-05 00:00:00.000000000
999997,Air Freshener,Mobile Payment,Atlanta,Department Store,Professional,Spring,Discount on Selected Items,False,N1000032238,1714262400000000000,Donna Lester,Peanut Butter,16:46:06,10,98.89,5,2176-09-26 00:00:00.000000000
999998,Air Freshener,Credit Card,Atlanta,Convenience Store,Teenager,Winter,No_promo,False,N1000744322,1608249600000000000,Matthew Harris,Shaving Cream,11:48:35,3,26.08,2,2020-12-29 00:00:00.000000000


In [38]:
synth_df.columns

Index(['Product', 'PaymentMethod', 'City', 'StoreType', 'CustomerCategory',
       'Season', 'Promotion', 'DiscountApplied', 'TransactionID', 'Date',
       'CustomerName', 'Product', 'time', 'TotalItems', 'TotalCost', 'Numprod',
       'Date'],
      dtype='object')

In [43]:
synth_df.to_csv('syn_data/syn_'+name, index=False)